# Matrix Factorization (ALS) for Implicit Feedback

Цели:
- построить рекомендательную модель на основе матричной факторизации (ALS)
  для implicit feedback (просмотры/корзина/покупки);
- получить персональные рекомендации на уровне user–item индексов;
- оценить качество по `precision@k` и `recall@k` и сравнить с popularity baseline.


In [57]:
import pandas as pd
import numpy as np

from scipy.sparse import coo_matrix
import implicit
import logging

logging.getLogger("implicit").setLevel(logging.WARNING)

usecols = ["visitorid", "itemid", "event", "timestamp"]

train = pd.read_csv("../data/processed/train.csv", usecols=usecols)
test = pd.read_csv("../data/processed/test.csv", usecols=usecols)

for df in (train, test):
    df["timestamp"] = pd.to_datetime(df["timestamp"])

train.shape, test.shape, train["event"].value_counts()


((1008982, 4),
 (126123, 4),
 event
 view           945976
 addtocart       46036
 transaction     16970
 Name: count, dtype: int64)

In [58]:
event_weight = {
    "view": 1.0,
    "addtocart": 2.0,
    "transaction": 3.0,
}

train["weight"] = train["event"].map(event_weight)
train["weight"].describe()


count    1.008982e+06
mean     1.079264e+00
std      3.265260e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      3.000000e+00
Name: weight, dtype: float64

In [59]:
user_ids = train["visitorid"].unique()
item_ids = train["itemid"].unique()

n_users = len(user_ids)
n_items = len(item_ids)

n_users, n_items


(161820, 74904)

In [60]:
user_id_to_idx = {uid: i for i, uid in enumerate(user_ids)}
item_id_to_idx = {iid: j for j, iid in enumerate(item_ids)}


In [61]:
# индексы в терминах матрицы
user_idx = train["visitorid"].map(user_id_to_idx).values
item_idx = train["itemid"].map(item_id_to_idx).values
data = train["weight"].astype(float).values

user_item = coo_matrix(
    (data, (user_idx, item_idx)),
    shape=(n_users, n_items),
).tocsr()

item_user = user_item.T.tocsr()  # (items x users) — то, что нужно implicit
user_item, item_user


(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 601486 stored elements and shape (161820, 74904)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 601486 stored elements and shape (74904, 161820)>)

In [62]:
factors = 64
regularization = 0.01
iterations = 15

als_model = implicit.als.AlternatingLeastSquares(
    factors=factors,
    regularization=regularization,
    iterations=iterations,
    random_state=42,
)

als_model.fit(item_user)


  0%|          | 0/15 [00:00<?, ?it/s]

In [63]:
import numpy as np

def recommend_als_idx(
    user_idx: int,
    model: implicit.als.AlternatingLeastSquares,
    user_item_matrix,
    N: int = 20,
) -> list[int]:
    """
    Возвращает список индексов товаров (item_idx) для пользователя (user_idx).
    Всё в индексах, без оригинальных ID.
    """
    num_users_model = model.user_factors.shape[0]
    if user_idx < 0 or user_idx >= num_users_model:
        # у модели просто нет факторов для такого пользователя
        return []

    # вектор пользователя и факторы товаров
    user_vec = model.user_factors[user_idx]        # (factors,)
    item_factors = model.item_factors             # (n_items, factors)

    scores = item_factors @ user_vec              # (n_items,)

    # не рекомендуем уже взаимодействованные товары
    known_items = user_item_matrix[user_idx].indices
    scores = scores.copy()
    scores[known_items] = -np.inf

    # если вдруг всё -inf (теоретически)
    valid_mask = np.isfinite(scores)
    if not valid_mask.any():
        return []

    # выбираем топ-N по score
    N_eff = min(N, valid_mask.sum())
    top_idx = np.argpartition(scores, -N_eff)[-N_eff:]
    top_idx = top_idx[np.argsort(scores[top_idx])[::-1]]

    return [int(i) for i in top_idx]


In [64]:
some_uidx = 0  # первый пользователь по индексу
recommend_als_idx(some_uidx, als_model, user_item, N=5)


[79224, 79513, 7153, 79795, 4932]

In [65]:
test_trans = test[test["event"] == "transaction"].copy()

# оставляем только пользователей и товары, которые есть в train-матрице
test_trans = test_trans[
    test_trans["visitorid"].isin(user_id_to_idx)
    & test_trans["itemid"].isin(item_id_to_idx)
].copy()

test_trans["uidx"] = test_trans["visitorid"].map(user_id_to_idx)
test_trans["iidx"] = test_trans["itemid"].map(item_id_to_idx)

len(test_trans), test_trans.head()


(509,
                    timestamp  visitorid        event  itemid    uidx   iidx
 1065 2015-09-01 04:49:58.475     361291  transaction  214461   49610  48375
 1664 2015-09-01 07:52:14.411     569740  transaction  350525  138041   6832
 2337 2015-09-01 14:57:33.098     582525  transaction     147  147304   8099
 2338 2015-09-01 14:57:33.099     582525  transaction  250462  147304  74604
 2431 2015-09-01 15:19:22.887     530559  transaction  207865   73867   3569)

In [66]:
test_useridx2itemsidx = (
    test_trans
    .groupby("uidx")["iidx"]
    .apply(set)
)

len(test_useridx2itemsidx)


140

In [67]:
from typing import Iterable, Set, Tuple

def precision_recall_at_k(
    recommended: Iterable[int],
    relevant: Set[int],
    k: int,
) -> Tuple[float, float]:
    if k == 0:
        return 0.0, 0.0

    rec_k = list(recommended)[:k]
    rec_set = set(rec_k)

    hit_count = len(rec_set & relevant)
    precision = hit_count / k
    recall = hit_count / len(relevant) if relevant else 0.0

    return precision, recall


In [68]:
def evaluate_als_idx(
    useridx2itemsidx: pd.Series,
    model: implicit.als.AlternatingLeastSquares,
    user_item_matrix,
    ks=(5, 10, 20),
) -> pd.DataFrame:
    rows = []

    num_users_model = model.user_factors.shape[0]

    for k in ks:
        precisions = []
        recalls = []

        for uidx, relevant_items in useridx2itemsidx.items():
            # если у модели нет факторов для этого пользователя — пропускаем
            if uidx < 0 or uidx >= num_users_model:
                continue

            recs = recommend_als_idx(
                user_idx=uidx,
                model=model,
                user_item_matrix=user_item_matrix,
                N=k,
            )

            p, r = precision_recall_at_k(recs, relevant_items, k)
            precisions.append(p)
            recalls.append(r)

        rows.append({
            "k": k,
            "precision@k": np.mean(precisions) if precisions else 0.0,
            "recall@k": np.mean(recalls) if recalls else 0.0,
        })

    return pd.DataFrame(rows)


In [69]:
als_results = evaluate_als_idx(
    useridx2itemsidx=test_useridx2itemsidx,
    model=als_model,
    user_item_matrix=user_item,
    ks=(5, 10, 20),
)

als_results


,k,precision@k,recall@k
0,5,0.0,0.0
1,10,0.0,0.0
2,20,0.0,0.0


In [70]:
# Возьмём пару пользователей из теста и вручную посмотрим пересечения
for uidx, relevant in list(test_useridx2itemsidx.items())[:5]:
    recs = recommend_als_idx(uidx, als_model, user_item, N=20)
    inter = set(recs) & relevant
    print(f"user_idx={uidx}, relevant={len(relevant)}, hits={len(inter)}")
    print("  relevant:", list(relevant)[:10])
    print("  recs   :", recs[:10])
    print("  intersect:", inter)
    print("-" * 40)


user_idx=2216, relevant=3, hits=0
  relevant: [36395, 25996, 6079]
  recs   : [89954, 7147, 945, 119188, 123162, 101523, 105931, 4899, 149531, 112839]
  intersect: set()
----------------------------------------
user_idx=2574, relevant=5, hits=0
  relevant: [2466, 54501, 31729, 55702, 11581]
  recs   : [23725, 396, 42093, 791, 61042, 43818, 5057, 50, 3334, 131106]
  intersect: set()
----------------------------------------
user_idx=4514, relevant=1, hits=0
  relevant: [26238]
  recs   : [12818, 4932, 4622, 4655, 36448, 30229, 9243, 145017, 4853, 31919]
  intersect: set()
----------------------------------------
user_idx=5033, relevant=1, hits=0
  relevant: [6231]
  recs   : [68676, 4974, 40526, 5702, 107984, 56642, 120601, 20145, 9004, 70567]
  intersect: set()
----------------------------------------
user_idx=5057, relevant=5, hits=0
  relevant: [5457, 2783, 24756, 4791, 43199]
  recs   : [68676, 131743, 105772, 945, 79795, 4932, 155744, 5744, 33541, 141940]
  intersect: set()
--------

## Выводы по ALS (implicit feedback)

В рамках эксперимента была обучена модель матричной факторизации
(Alternating Least Squares) на implicit feedback матрице user–item.
Веса сигналов:
- `view` — 1.0
- `addtocart` — 2.0
- `transaction` — 3.0

Для оценки качества мы использовали только покупки в test-выборке
(`event == "transaction"`) как релевантные объекты
и считали метрики `precision@k` и `recall@k`.

В текущей конфигурации модели значения `precision@k` и `recall@k` оказались равны 0
для всех рассмотренных `k`. Это означает, что ни одна из топ-N рекомендаций ALS
не совпала с товарами, реально купленными пользователями в test-периоде.

### Возможные причины

- Очень разреженные данные: в test-выборке всего 509 покупок на 140 пользователей.
- Модель обучалась на всех типах событий (view / addtocart / transaction),
  но метрики считались только на покупках. ALS мог подстроиться под общие паттерны
  просмотров и не воспроизводить конкретные купленные товары.
- Гиперпараметры (число факторов, регуляризация, количество итераций)
  не подбирались, использовались базовые значения.
- Фильтрация already-seen товаров при построении рекомендаций уменьшает шанс
  "угадывания" тех же товаров, что и в test.

### Итог

Данный эксперимент показывает, что на разреженных данных e-commerce
простая конфигурация ALS по implicit feedback может существенно уступать
даже простому popularity baseline. Это мотивирует:
- либо более тщательно подбирать гиперпараметры и схему весов,
- либо использовать альтернативные модели (например, обучаемое ранжирование),
- либо менять постановку задачи (например, предсказывать не только покупки,
  но и добавления в корзину или просмотры).
